# Julia is roughly as fast as C or Fortran

Julia's core innovation: language design optimized for just-in-time compilation
  1. strict type system
  2. type inference
  3. type stability
  4. multiple dispatch
  5. just-in-time compilation



## Just-in-time compilation 

Define a simple function and run it twice

In [1]:
f(x) = x^3 - 2

f (generic function with 1 method)

In [2]:
@time f(0.3);
@time f(0.4);

  0.002877 seconds (1.20 k allocations: 74.862 KiB)
  0.000001 seconds (4 allocations: 160 bytes)


In [3]:
@time f(0.3)

  0.000001 seconds (4 allocations: 160 bytes)


-1.973

** The second evaluation is 3000 times faster than the first! Why? Just-in-time compilation.**

In [4]:
@code_lowered(f(7.0))   # show f(x) in Julia's abstract syntax tree

CodeInfo(:(begin 
        nothing
        return (Base.literal_pow)(Main.^, x, (Core.apply_type)(Base.Val, 3)) - 2
    end))

In [10]:
@code_typed(f(7.0))     # show f(x) in abstract syntax tree with types determined 

CodeInfo(:(begin 
        return (Base.sub_float)((Base.mul_float)((Base.mul_float)(x, x)::Float64, x)::Float64, (Base.sitofp)(Float64, 2)::Float64)::Float64
    end))=>Float64

In [5]:
@code_llvm f(7.0)       # show f(x) in LLVM (compiler) intermediate language


define double @julia_f_60982(double) #0 !dbg !5 {
top:
  %1 = fmul double %0, %0
  %2 = fmul double %1, %0
  %3 = fadd double %2, -2.000000e+00
  ret double %3
}


In [6]:
@code_native f(7.0)     # show f(x) in Intel IA-64 assembly language

	.text
Filename: In[1]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	movapd	%xmm0, %xmm1
	mulsd	%xmm1, %xmm1
	mulsd	%xmm0, %xmm1
	movabsq	$140320096917288, %rax  # imm = 0x7F9ED1881728
	addsd	(%rax), %xmm1
	movapd	%xmm1, %xmm0
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)


## Type inference

Julia figures out the types of untyped variables --crucial for compiling to machine code!

In [7]:
@code_llvm(f(7.0))


define double @julia_f_60982(double) #0 !dbg !5 {
top:
  %1 = fmul double %0, %0
  %2 = fmul double %1, %0
  %3 = fadd double %2, -2.000000e+00
  ret double %3
}


In [8]:
@code_llvm(f(7))


define i64 @julia_f_61094(i64) #0 !dbg !5 {
top:
  %1 = mul i64 %0, %0
  %2 = mul i64 %1, %0
  %3 = add i64 %2, -2
  ret i64 %3
}


## Type stability

Just-in-time compilation works best if types of temporaries and return values can be determined at compile-time. So Julia prefers to be strict about types.

In [12]:
sqrt(-1.0)           # no problem in Python or Matlab, but a problem in Julia

LoadError: DomainError:
sqrt will only return a complex result if called with a complex argument. Try sqrt(complex(x)).

In [14]:
sqrt(-1.0 + 0.0im)   # In Julia, if you want a complex sqrt, use the cmplex sqrt

0.0 + 1.0im

In [15]:
methods(sqrt)

# 10 methods for generic function "sqrt":
sqrt(x::BigInt) in Base.MPFR at mpfr.jl:486
sqrt(x::BigFloat) in Base.MPFR at mpfr.jl:477
sqrt(a::Complex{Float16}) in Base.Math at math.jl:951
sqrt(a::Float16) in Base.Math at math.jl:950
sqrt(x::Float32) in Base.Math at math.jl:426
sqrt(x::Float64) in Base.Math at math.jl:425
sqrt(z::Complex{#s45} where #s45<:AbstractFloat) in Base at complex.jl:392
sqrt(z::Complex) in Base at complex.jl:416
sqrt(x::Real) in Base.Math at math.jl:434
sqrt(x::AbstractArray{T,N} where N) where T<:Number in Base at deprecated.jl:56

## Comparison: millionth iterate of logistic map in Julia and C

Define $f(x) = 4x(1-x)$, generate millionth iterate function $f^N(x)$

In [16]:
# define function that, given an f, returns iterated function f^N
function iterator(f, N)
    
    # construct f^N
    function fN(x)
      for i ∈ 1:N             
        x = f(x)
      end
      x
    end    
    
    fN     # return f^N
end

# define logistic map function
f(x) = 4*x*(1-x)

# use iterator function to constuct millionth iterate of logistic map
fᴺ  = iterator(f, 1000000)  

(::fN) (generic function with 1 method)

In [17]:
@time x = fᴺ(0.34);
@time x = fᴺ(0.67);

  0.011285 seconds (1.79 k allocations: 100.256 KiB)
  0.003043 seconds (5 allocations: 176 bytes)


In [18]:
@show x;

x = 0.10116885334547539


### Equivalent C++ code

note: starting semicolon tells Julia to execute Unix shell code

In [19]:
; pwd

/home/gibson/gitworking/whyjulia


In [22]:
; cat fmillion.cpp

#include <stdlib.h>
#include <iostream>
#include <iomanip>
#include <ctime>

using namespace std;

double f(double x) {
  return 4*x*(1-x);
}

int main(int argc, char* argv[]) {
  double x = argc > 1 ? atof(argv[1]) : 0.0;

  double t0 = clock();
  for (int n=0; n<1000000; ++n)
    x = f(x);
  double t1 = clock();

  cout << "t = " << (t1-t0)/CLOCKS_PER_SEC << " seconds" << endl;
  cout << setprecision(17);
  cout << "x = " << x << endl;
  
  return 0;
}
  


In [23]:
; g++ -O3 -o fmillion fmillion.cpp

### Execution time for C++

In [24]:
; fmillion 0.67

t = 0.002746 seconds
x = 0.10116885334547539


### Execution time for Julia

In [25]:
print("t=")
@time x = fᴺ(0.67);
@show x;

t=  0.002496 seconds (5 allocations: 176 bytes)
x = 0.10116885334547539


Execution times $t$ are comparable. Sometimes Julia is faster, sometimes C. Millionth iterates $x$ are the same, indicate same sequence of floating-point operations. 

## Julia benchmarks versus other languages

Execution times are normalized so C=1.

![Julia benchmarks](juliabenchmarks.png  " Julia benchmarks")

http://julialang.org/benchmarks
